# PRONÓSTICO DEL AVALUO DE VIVIENDAS

## CAPITULO I. Reducción de la dimensionalidad y prealistamiento de datos
_____

Buscando lograr una mayor eficiencia en el proceso de pronóstico se realiza una limpieza preliminar de variables buscando incorporar aquellas  que pueden llegar a explicar de mejor manera el comportamiento del avaluó de viviendas.

### i) Variables que  que por criterio de negocio no se consideran dentro del análisis
- **Variables concentradas:** como objeto, motivo, tipo_credito se encuentran altamente concentradas en una categoria, por ello buscando reducir la complejidad del modelo se eliminan.

- **Variables texto:** casos como dirección_inmueble y descripciones aportan información valiosa pero en un primer acercamiento no se tendrán en cuenta.


> Un mayor detalle de las variables se encuentra en el archivo "Diccionario-variables.xlsx"


### ii) Unificación de variables
- **Areas sociales:** el tener porteria, piscina, biciletero se unifican en una sola variable llamada areas_sociales de tal suerte que tener mas de 8 áreas  sociales es excelente, entre 4 y 8 bueno y menor a 4 regular.
- **Numero de habitaciones, cocina y otros:** se unifican el numero de espacios del predio
- **Area construida:** es la suma de todas las areas de la vivienda
- **Estado terminados:** se unifican las variables de estado de acabados de la vivienda usando puntajes de acuerdo a la calidad de los acabados.

In [311]:
import pandas as pd
import numpy as np

In [312]:
#Cargue de datos ,instanciamiento de parametros generales y algunas funciones genericas
baseAvaluos = pd.read_csv("train_precios_vivienda.csv", sep = ",", dtype="str", encoding="utf-8")
variablesElegidas = pd.read_excel("Diccionario-variables.xlsx", sheet_name="Parametros")
SELECCION_VARIABLES = list(variablesElegidas['variables_incluye'].drop_duplicates().dropna())
baseAvaluos = baseAvaluos[SELECCION_VARIABLES]

CONSOLIDA_AREAS_SOCIALES = {
    "nombre_variable": "areas_sociales",
    "variables": ["porteria", "citofono", "bicicletero", "piscina", "tanque_de_agua",
                  "club_house", "garaje_visitantes", "teatrino", "sauna", 
                  "vigilancia_privada"],
    "umbrales":{
        "e": 8,
        "b": 4,
        "r": 0
    }
}

CONSOLIDA_VARIABLES = {
    "areas_sociales": {
        "variables": ["porteria", "citofono", "bicicletero", "piscina", "tanque_de_agua",
                  "club_house", "garaje_visitantes", "teatrino", "sauna", 
                  "vigilancia_privada"],
        "criterios":{"SI":1},
        "umbrales":{"e": 8, "b": 4, "r": 0}    
    },
    "estado_terminados":{
        "variables" : ["estado_acabados_pisos", "estado_acabados_muros", "estado_acabados_techos", "estado_acabados_madera",
                       "estado_acabados_metal", "estado_acabados_banos", "estado_acabados_cocina"],
        "criterios":{"LUJOSO":10, "BUENO":8, "NORMAL":5, "SENCILLO":5,"REGULAR":2},
        "umbrales":{"e": 60, "b": 45, "r": 25, "m": 0}
    },
    "luz_iluminacion":{
        "variables" :["iluminacion", "ventilacion"],
        "criterios":{"BUENO":10, "REGULAR":5},
        "umbrales":{"e": 15, "b": 10, "r": 0}
    }
     
}
SUMA_VARIABLES ={
    "numero_habitaciones":["habitaciones", "estar_habitacion","cuarto_servicio"],
    "numero_banos":["bano_privado", "bano_social", "bano_servicio"],
    "otros_espacios":["estudio", "balcon", "terraza", "patio_interior", "jardin"],
    "area_construida":["area_privada","area_garaje","area_deposito", "area_construccion", "area_otros", "area_terreno", "area_libre"],
        
}
VARIABLES_TRANSFORMA_NUMERO = ["valor_total_avaluo", "area_valorada"]

GUIA_USO_COMERCIAL = ["INDUSTRI","COMERC"]

def transformaNumero(baseOrigen: pd.DataFrame, variablesUsar: list) -> pd.DataFrame:
    """
    Funcion que transforma las variables de un dataframe en numericas siguiendo algunas reglas de reemplazo de caracteres
    string que no son numericos
     
    """
    baseEspecifica = baseOrigen[variablesUsar].copy().fillna(0)
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(r"[^0-9.,]", "",regex = True) #que no contenga digitos o punto reemplaza por ""
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(r"[,]", ".",regex = True) #reemplaza comas por puntos
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(r"\.{2,}", ".",regex = True) #dos puntos seguidos reemplaza por uno solo
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace("","0")
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(".","0")
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(".0.0","0")
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(".10.","0")
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(".5.","0")
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].replace(".2010.","0")
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].astype(float)
    
    return baseEspecifica

In [313]:
def construyeVariablesCombinadas(baseOrigen: pd.DataFrame, variablesCombinadas:dict) -> pd.DataFrame:
    baseCalculo = baseOrigen.copy()
    #Itera sobre las variables nuevas a crear
    for nuevaVar in variablesCombinadas.keys():
        variablesCombinar = variablesCombinadas[nuevaVar]["variables"]
        criteriosCorte = variablesCombinadas[nuevaVar]["criterios"]
        umbrales = variablesCombinadas[nuevaVar]["umbrales"]
        baseVariablesCombinar = baseCalculo[variablesCombinar].copy()
        #itera sobre cada variable que compone la variable combinada
        for variable in variablesCombinar:
            baseVariablesCombinar[variable + "_tmp"] = 0
            #asigna un puntaje dependiendo del criterio
            for criterio in criteriosCorte.keys():
                baseVariablesCombinar[variable + "_tmp"] = np.where(
                    baseVariablesCombinar[variable].str.upper() == criterio,
                    criteriosCorte[criterio],
                    baseVariablesCombinar[variable + "_tmp"]
                )
            baseVariablesCombinar[variable] = baseVariablesCombinar[variable + "_tmp"]
        
        #realiza la suma de los puntajes asignados a cada variable
        baseVariablesCombinar[nuevaVar + "_numero"] = baseVariablesCombinar[variablesCombinar].sum(axis=1)
        
        #asigna un rango dependiendo de la suma de los puntajes
        baseVariablesCombinar[nuevaVar] = ""
        for umbral in umbrales:
            baseVariablesCombinar[nuevaVar] = np.where(
                ((baseVariablesCombinar[nuevaVar + "_numero"] >= umbrales[umbral]) &
                 (baseVariablesCombinar[nuevaVar] == "")), 
                umbral, 
                baseVariablesCombinar[nuevaVar]
            )
        #elimina las variables que se combinaron
        baseCalculo.drop(variablesCombinar, axis=1, inplace=True)
        baseCalculo = pd.concat([baseCalculo, baseVariablesCombinar[nuevaVar]], axis=1)
        
        print(f"Variable {nuevaVar} creada")
    return baseCalculo
      
baseAvaluos = construyeVariablesCombinadas(baseOrigen = baseAvaluos, variablesCombinadas = CONSOLIDA_VARIABLES)

Variable areas_sociales creada
Variable estado_terminados creada
Variable luz_iluminacion creada


In [314]:
#construye variables que son el resultado de sumar otras
def construyeVariablesSuma(baseOrigen: pd.DataFrame, variablesSuma:dict) -> pd.DataFrame:
    baseCalculo = baseOrigen.copy()
    for nuevaVar in variablesSuma.keys():
        variablesUsar = variablesSuma[nuevaVar]
        
        baseEspecifica = transformaNumero(baseOrigen = baseCalculo, variablesUsar = variablesUsar)
        baseEspecifica[nuevaVar] = baseEspecifica[variablesUsar].sum(axis=1)
        
        baseOrigen = pd.concat([baseOrigen, baseEspecifica[nuevaVar]], axis=1)
        baseOrigen.drop(variablesUsar, axis=1, inplace=True)
        print(f"Variable {nuevaVar} creada")
    return baseOrigen

baseAvaluos = construyeVariablesSuma(baseOrigen = baseAvaluos, variablesSuma = SUMA_VARIABLES)

Variable numero_habitaciones creada
Variable numero_banos creada
Variable otros_espacios creada
Variable area_construida creada


In [315]:
#transforma algunas variables a numericas
def transformaNumericas(baseOrigen: pd.DataFrame, variablesUsar: list) -> pd.DataFrame:
    baseEspecifica = transformaNumero(baseOrigen = baseOrigen, variablesUsar = variablesUsar)
    baseEspecifica[variablesUsar] = baseEspecifica[variablesUsar].astype(float)
    
    for variable in variablesUsar:
        baseOrigen[variable] = baseEspecifica[variable]
    print(f"Variables {variablesUsar} transformadas a numericas")
    return baseOrigen

baseAvaluos = transformaNumericas(baseOrigen = baseAvaluos, variablesUsar = VARIABLES_TRANSFORMA_NUMERO)

Variables ['valor_total_avaluo', 'area_valorada'] transformadas a numericas


In [316]:
#crea variable para determinar si es una vivienda residencial o comercial
def determinaTipoVivienda(baseOrigen: pd.DataFrame) -> pd.DataFrame:
    baseCalculo = baseOrigen.copy()
    variablesNumericas = ["local", "oficina", "bodega"]
    baseCalculo["suma_comercial"] = baseCalculo[variablesNumericas].sum(axis=1)
    baseCalculo["tipo_uso_inmueble"] = ""
    baseCalculo["tipo_uso_inmueble"] = np.where(
        baseCalculo["suma_comercial"] > 0,
        "C",
        ""
    )
    
    baseCalculo["tipo_vivienda"] = np.where(
        baseCalculo["tipo_inmueble"].str.upper() == "RESIDENCIAL",
        "R",
        "C"
    )
    baseCalculo.drop("tipo_inmueble", axis=1, inplace=True)
    print("Variable tipo_vivienda creada")
    return baseCalculo

In [318]:
baseAvaluos["area_actividad"].unique()

array(['0', 'Vivienda', 'RESIDENCIAL', 'Residencial', 'residencial',
       'Mixta', 'VIVIENDA', 'Residencial neta', 'mixta', 'MULTIPLE',
       '47.8', 'Residencial Predominante', 'Residencial y comercial',
       'RP', '60,60', '48', '91', 'MIXTA', 'ÁREA RESIDENCIAL TURISTI',
       'Residencial 1', 'Rsidencial', 'Residencial y turistica',
       'Reisdencial', 'Residencia', 'Habitacional', 'ar-4',
       'Plan Zonal rural', '72', '56.9', 'Residencial - TurÃ\xadstico',
       'Rediencial', 'R-1 Control PaisajÃ\xadstico', '3,3', '80', '98.49',
       'R-2', 'residencia', '59.76', '35,01', '67.29', 'BAJA MIXTURA',
       'vVvienda', '103', 'vivienda', '10', 'Residual', '38.04',
       'RESIDENCIA', '65.33', '1', '84.75', 'PR-1B', 'Residencial Neto',
       'R-1', 'RESIDENCIL', 'MIXTO', 'Residencial y Comercial',
       'Residencial predominante', '51.56', 'RESDENCIAL', '57.60', '66.8',
       'Reidencial', 'Exterior', 'RESIDENCIAL.', 'Residencial Neta',
       'Residencial Tipo C', '92